### Bart_medical Train model

In [2]:
pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00


In [3]:
!pip install evaluate
# Installs the 'evaluate' library using pip.

# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration            # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments               # fine-tune model
from transformers import pipeline                                               # pipeline
from transformers import DataCollatorForSeq2Seq                                 # DataCollator to batch the data
from transformers import EarlyStoppingCallback
import torch                                                                    # PyTorch
import evaluate                                                                 # Hugging Face's library for model evaluation
from datasets import Dataset

import numpy as np
import pandas as pd
import nltk
import html
# from textblob import TextBlob                                                 # fix spelling mistakes in texts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split                   # identify the most common terms in the corpus
import re                                                                       # clean text data
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available - Using GPU")
    device = torch.device('cuda')
else:
    print("GPU is not available - Using CPU")
    device = torch.device('cpu')

GPU is available - Using GPU


In [5]:
pd.set_option('display.max_colwidth', 1000)

In [6]:
#Provide the file_path
file_path = 'Datasets/medical_samsum/New_medical_meadow_cord19 (1).csv'

In [7]:
def display_feature_list(features, feature_type):

    '''
    This function displays the features within each list for each type of data
    '''

    print(f"\n{feature_type} Features: ")
    print(', '.join(features) if features else 'None')

def describe_df(df):

    global categorical_features
    categorical_features = [col for col in df.columns if df[col].dtype == 'object']

    print(f"\n{type(df).__name__} shape: {df.shape}")
    print(f"\n{df.shape[0]:,.0f} samples")
    print(f'\nMissing Data: \n{df.isnull().sum()}')
    print(f'\nDuplicates: {df.duplicated().sum()}')

    display_feature_list(categorical_features, 'Categorical')

    print(f'\n{type(df).__name__} Head: \n')
    display(df.head(5))
    print(f'\n{type(df).__name__} Tail: \n')
    display(df.tail(5))

In [8]:
df = pd.read_csv(file_path)

In [9]:
df.head()

,instruction,input,output
0,Please summerize the given abstract to a title,"INTRODUCTION: Infection with the new coronavirus responsible for Severe Acute Respiratory Syndrome (SARS-CoV-2) continues to spread worldwide. In Brazil, there are already more than 230 thousand dead, many of these older adults. OBJECTIVE: To present the clinical characteristics of older Brazilian adults infected by COVID-19, in the epidemiological weeks (EW) 34-52, and to verify factors responsible for the increased risk of death. METHODS: Retrospective and observational study conducted with secondary publicly available data, provided by the Brazilian Ministry of Health. 1,544 confirmed cases of registered COVID-19 infection were included between August 16 and December 26, 2020, aged 60 or older. OUTCOMES: Demographic data, comorbidity, symptoms for disease, clinical information: days of hospitalization, chest X-ray, type of RT-PCR. RESULTS: 48% of patients admitted to the ICU with evidence for SARS-CoV-2 died. Symptoms and comorbidities related to increased chance of death (OR) w...",Clinical characteristics of 1544 Brazilians aged 60 years and over with laboratory evidence for SARS-CoV-2
1,Please summerize the given abstract to a title,"OBJECTIVE: To compare maternal and perinatal outcomes between randomized trials and observational studies in which conservative management was performed for more than 48 h in patients with early-onset severe preeclampsia. METHODOLOGY: We searched PubMed, LILACS, Cochrane and Google Scholar. The studies were divided in two groups: randomized and observational studies, from 1990 to 2018 that included patients with severe preeclampsia before 34 weeks of gestation with pregnancy prolongation &#8805;48 h but that did not include fetal growth restriction or HELLP syndrome at the beginning. The main variables recorded were maternal and perinatal complications. MAIN RESULTS: Forty-four studies met the inclusion criteria, and 5 of these were randomized. The average pregnancy prolongation was 9 days, with no difference between groups. Maternal complications were significantly more common in observational studies, RR = 0.71, 95% CI (0.54-0.93), p = .009. Perinatal complications were also sign...",Conservative management of early-onset severe preeclampsia: comparison between randomized and observational studies a systematic review
2,Please summerize the given abstract to a title,"Background:In recent years, an integrated programme of vector control, screening and treatment of gambiense human African trypanosomiasis (gHAT) infections has led to a rapid decline in cases in the Mandoul disease focus of Chad. In this study, we assess whether elimination of transmission has already been achieved in the region despite low-level case reporting, quantify the role of intensified interventions in transmission reduction, and predict the trajectory of gHAT in Mandoul for the next decade under a range of control scenarios. Method: We utilise human case data (2000-2019) to update a previous model of transmission of gHAT in Mandoul. We also test the updated model, which now has refined assumptions on diagnostic specificity of the current algorithm and an improved fitting method, via a data censoring approach.Results:We conclude that passive detection rates have increased due to improvements in diagnostic availability in fixed health facilities since 2015, by 2.1-fold for ...","Update of transmission modelling and projections of gambiense human African trypanosomiasis in the Mandoul focus, Chad"
3,Please summerize the given abstract to a title,"Microanatomy of the vast majority of human organs at birth is characterized by marked differences as compared to adult organs, regarding their architecture and the cell types detectable at histology. In preterm neonates, these differences are even more evident, due to the lower level of organ maturation and to ongoing cell differentiation. One of the most remarkable finding in preterm tissues is the pres

In [10]:
# Get column names
column_names_list = df.columns.tolist()
print(column_names_list)


['instruction', 'input', 'output']


In [11]:
# Drop a column
df = df.drop(['instruction'],axis=1)


In [12]:
# Extracting info on the Dataframe
describe_df(df)


DataFrame shape: (821007, 2)

821,007 samples

Missing Data: 
input     2
output    0
dtype: int64

Duplicates: 65789

Categorical Features: 
input, output

DataFrame Head: 



,input,output
0,"INTRODUCTION: Infection with the new coronavirus responsible for Severe Acute Respiratory Syndrome (SARS-CoV-2) continues to spread worldwide. In Brazil, there are already more than 230 thousand dead, many of these older adults. OBJECTIVE: To present the clinical characteristics of older Brazilian adults infected by COVID-19, in the epidemiological weeks (EW) 34-52, and to verify factors responsible for the increased risk of death. METHODS: Retrospective and observational study conducted with secondary publicly available data, provided by the Brazilian Ministry of Health. 1,544 confirmed cases of registered COVID-19 infection were included between August 16 and December 26, 2020, aged 60 or older. OUTCOMES: Demographic data, comorbidity, symptoms for disease, clinical information: days of hospitalization, chest X-ray, type of RT-PCR. RESULTS: 48% of patients admitted to the ICU with evidence for SARS-CoV-2 died. Symptoms and comorbidities related to increased chance of death (OR) w...",Clinical characteristics of 1544 Brazilians aged 60 years and over with laboratory evidence for SARS-CoV-2
1,"OBJECTIVE: To compare maternal and perinatal outcomes between randomized trials and observational studies in which conservative management was performed for more than 48 h in patients with early-onset severe preeclampsia. METHODOLOGY: We searched PubMed, LILACS, Cochrane and Google Scholar. The studies were divided in two groups: randomized and observational studies, from 1990 to 2018 that included patients with severe preeclampsia before 34 weeks of gestation with pregnancy prolongation &#8805;48 h but that did not include fetal growth restriction or HELLP syndrome at the beginning. The main variables recorded were maternal and perinatal complications. MAIN RESULTS: Forty-four studies met the inclusion criteria, and 5 of these were randomized. The average pregnancy prolongation was 9 days, with no difference between groups. Maternal complications were significantly more common in observational studies, RR = 0.71, 95% CI (0.54-0.93), p = .009. Perinatal complications were also sign...",Conservative management of early-onset severe preeclampsia: comparison between randomized and observational studies a systematic review
2,"Background:In recent years, an integrated programme of vector control, screening and treatment of gambiense human African trypanosomiasis (gHAT) infections has led to a rapid decline in cases in the Mandoul disease focus of Chad. In this study, we assess whether elimination of transmission has already been achieved in the region despite low-level case reporting, quantify the role of intensified interventions in transmission reduction, and predict the trajectory of gHAT in Mandoul for the next decade under a range of control scenarios. Method: We utilise human case data (2000-2019) to update a previous model of transmission of gHAT in Mandoul. We also test the updated model, which now has refined assumptions on diagnostic specificity of the current algorithm and an improved fitting method, via a data censoring approach.Results:We conclude that passive detection rates have increased due to improvements in diagnostic availability in fixed health facilities since 2015, by 2.1-fold for ...","Update of transmission modelling and projections of gambiense human African trypanosomiasis in the Mandoul focus, Chad"
3,"Microanatomy of the vast majority of human organs at birth is characterized by marked differences as compared to adult organs, regarding their architecture and the cell types detectable at histology. In preterm neonates, these differences are even more evident, due to the lower level of organ maturation and to ongoing cell differentiation. One of the most remarkable finding in preterm tissues is the presence of huge amounts of stem/progenitor cells in multiple organs, including kidney, brain, heart, adrenals, and lungs. In other organs, such as liver, the completely different burden of cell types in 


DataFrame Tail: 



,input,output
821002,"The Coronavirus Disease 2019 (COVID-19) pandemic has become the worst pandemic in modern history. The lack of prior immunity to the virus has resulted in a high mortality rate, though children have fared better than adults, overall. We present a case of a child who developed B-cell acute lymphoblastic leukemia 1 week following a symptomatic COVID-19 infection. It is possible that this viral infection provided the ""second hit"" posited to occur in pediatric leukemogenesis as proposed by Dr Greaves, with his initial viral exposure occurring several weeks earlier.","Childhood B-Cell Acute Lymphoblastic Leukemia Following SARS CoV-2 Infection: A Potential Second ""Hit"" in Leukemogenesis"
821003,"AIM: To investigate the effects of years of nursing experience and mental health on work impairment among nurses during the COVID‐19 crisis. BACKGROUND: During the COVID‐19 crisis, nurses experience a greater psychological burden than other health care workers. Studies have not yet investigated the effects of years in nursing and mental health on potential work impairment during the COVID‐19 crisis in nurses. METHODS: A survey was administered to 83 nurses on active duty during the COVID‐19 crisis. The graphical LASSO and the DAG helped estimate the associations between years of nursing experience, mental health and work impairment. RESULTS: A moderate negative correlation emerged between years of nursing experience, avoidance and work impairment. A direct effect was observed between anxiety and work impairment. A moderate positive correlation emerged between anxiety, depression and work impairment. An indirect effect was observed between depression, burnout, insomnia, years of nur...",Nursing professionals' mental well‐being and workplace impairment during the COVID‐19 crisis: A Network analysis
821004,"Laboratory diagnosis of Lyme neuroborreliosis (LNB) is challenging, and validated diagnostic algorithms are lacking. Therefore, this retrospective cross-sectional study aimed to compare the diagnostic performance of seven commercial antibody assays for LNB diagnosis. Random forest (RF) modeling was conducted to investigate whether the diagnostic performance using the antibody assays could be improved by including several routine cerebrospinal fluid (CSF) parameters (i.e., leukocyte count, total protein, blood-CSF barrier functionality, and intrathecal total antibody synthesis), two-tier serology on serum, the CSF level of the B-cell chemokine (C-X-C motif) ligand 13 (CXCL13), and a Borrelia species PCR on CSF. In total, 156 patients were included who were classified as definite LNB (n = 10), possible LNB (n = 7), or non-LNB patient (n = 139) according to the criteria of the European Federation of Neurological Societies using a consensus strategy for intrathecal Borrelia-specific an...",Retrospective Evaluation of Various Serological Assays and Multiple Parameters for Optimal Diagnosis of Lyme Neuroborreliosis in a Routine Clinical Setting.
821005,"Background/AimsArterial thrombosis and positive antiphospholipid antibodies havebeen reported following COVID-19 infection. We report a case of aorticthrombosis occurring after COVID-19 pneumonitis, with persistentlypositive anticardiolipin antibodies.MethodsA 63-year-old lady presented with sudden onset of abdominal painradiating to the back, associated with vomiting. She reported feelingslightly breathless, and ten days prior to admission a throat swab forSARS-CoV-2 tested positive. Oxygen saturation was 96% (94-100%)on room air. Respiratory examination revealed minimal basal cracklesbilaterally. Her heart sounds were normal, and there was no carotid orsubclavian bruit. Radial, ulnar, brachial, subclavian and temporalpulses were intact and symmetrical. The abdomen was soft, but tenderover the right upper quadrant and epigastrium. Nailfolds were normal, and there was no livedo reticularis and no vasculitic rash. Initialinvestigations showed a CRP of 235 mg/L (&lt;5), Whit

In [13]:
# Drop duplicate rows
df_cleaned = df.drop_duplicates()

# Display DataFrame after dropping duplicates
print("\nDataFrame after dropping duplicates:")
print(df_cleaned)


DataFrame after dropping duplicates:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [14]:
# FInd the null inputs
NullRaws = df_cleaned['input'].isnull()
filtered_df = df_cleaned[NullRaws]
filtered_df

,input,output
297403,NaN,Lung transplantation in the COVID-19 Era: A multi-faceted challenge
699631,NaN,Primary Care Practitioners' Response to 2019 Novel Coronavirus Outbreak in China


In [15]:
# removing all null values
df_cleaned = df_cleaned.dropna()

In [16]:
describe_df(df_cleaned)


DataFrame shape: (755216, 2)

755,216 samples

Missing Data: 
input     0
output    0
dtype: int64

Duplicates: 0

Categorical Features: 
input, output

DataFrame Head: 



,input,output
0,"INTRODUCTION: Infection with the new coronavirus responsible for Severe Acute Respiratory Syndrome (SARS-CoV-2) continues to spread worldwide. In Brazil, there are already more than 230 thousand dead, many of these older adults. OBJECTIVE: To present the clinical characteristics of older Brazilian adults infected by COVID-19, in the epidemiological weeks (EW) 34-52, and to verify factors responsible for the increased risk of death. METHODS: Retrospective and observational study conducted with secondary publicly available data, provided by the Brazilian Ministry of Health. 1,544 confirmed cases of registered COVID-19 infection were included between August 16 and December 26, 2020, aged 60 or older. OUTCOMES: Demographic data, comorbidity, symptoms for disease, clinical information: days of hospitalization, chest X-ray, type of RT-PCR. RESULTS: 48% of patients admitted to the ICU with evidence for SARS-CoV-2 died. Symptoms and comorbidities related to increased chance of death (OR) w...",Clinical characteristics of 1544 Brazilians aged 60 years and over with laboratory evidence for SARS-CoV-2
1,"OBJECTIVE: To compare maternal and perinatal outcomes between randomized trials and observational studies in which conservative management was performed for more than 48 h in patients with early-onset severe preeclampsia. METHODOLOGY: We searched PubMed, LILACS, Cochrane and Google Scholar. The studies were divided in two groups: randomized and observational studies, from 1990 to 2018 that included patients with severe preeclampsia before 34 weeks of gestation with pregnancy prolongation &#8805;48 h but that did not include fetal growth restriction or HELLP syndrome at the beginning. The main variables recorded were maternal and perinatal complications. MAIN RESULTS: Forty-four studies met the inclusion criteria, and 5 of these were randomized. The average pregnancy prolongation was 9 days, with no difference between groups. Maternal complications were significantly more common in observational studies, RR = 0.71, 95% CI (0.54-0.93), p = .009. Perinatal complications were also sign...",Conservative management of early-onset severe preeclampsia: comparison between randomized and observational studies a systematic review
2,"Background:In recent years, an integrated programme of vector control, screening and treatment of gambiense human African trypanosomiasis (gHAT) infections has led to a rapid decline in cases in the Mandoul disease focus of Chad. In this study, we assess whether elimination of transmission has already been achieved in the region despite low-level case reporting, quantify the role of intensified interventions in transmission reduction, and predict the trajectory of gHAT in Mandoul for the next decade under a range of control scenarios. Method: We utilise human case data (2000-2019) to update a previous model of transmission of gHAT in Mandoul. We also test the updated model, which now has refined assumptions on diagnostic specificity of the current algorithm and an improved fitting method, via a data censoring approach.Results:We conclude that passive detection rates have increased due to improvements in diagnostic availability in fixed health facilities since 2015, by 2.1-fold for ...","Update of transmission modelling and projections of gambiense human African trypanosomiasis in the Mandoul focus, Chad"
3,"Microanatomy of the vast majority of human organs at birth is characterized by marked differences as compared to adult organs, regarding their architecture and the cell types detectable at histology. In preterm neonates, these differences are even more evident, due to the lower level of organ maturation and to ongoing cell differentiation. One of the most remarkable finding in preterm tissues is the presence of huge amounts of stem/progenitor cells in multiple organs, including kidney, brain, heart, adrenals, and lungs. In other organs, such as liver, the completely different burden of cell types in 


DataFrame Tail: 



,input,output
821002,"The Coronavirus Disease 2019 (COVID-19) pandemic has become the worst pandemic in modern history. The lack of prior immunity to the virus has resulted in a high mortality rate, though children have fared better than adults, overall. We present a case of a child who developed B-cell acute lymphoblastic leukemia 1 week following a symptomatic COVID-19 infection. It is possible that this viral infection provided the ""second hit"" posited to occur in pediatric leukemogenesis as proposed by Dr Greaves, with his initial viral exposure occurring several weeks earlier.","Childhood B-Cell Acute Lymphoblastic Leukemia Following SARS CoV-2 Infection: A Potential Second ""Hit"" in Leukemogenesis"
821003,"AIM: To investigate the effects of years of nursing experience and mental health on work impairment among nurses during the COVID‐19 crisis. BACKGROUND: During the COVID‐19 crisis, nurses experience a greater psychological burden than other health care workers. Studies have not yet investigated the effects of years in nursing and mental health on potential work impairment during the COVID‐19 crisis in nurses. METHODS: A survey was administered to 83 nurses on active duty during the COVID‐19 crisis. The graphical LASSO and the DAG helped estimate the associations between years of nursing experience, mental health and work impairment. RESULTS: A moderate negative correlation emerged between years of nursing experience, avoidance and work impairment. A direct effect was observed between anxiety and work impairment. A moderate positive correlation emerged between anxiety, depression and work impairment. An indirect effect was observed between depression, burnout, insomnia, years of nur...",Nursing professionals' mental well‐being and workplace impairment during the COVID‐19 crisis: A Network analysis
821004,"Laboratory diagnosis of Lyme neuroborreliosis (LNB) is challenging, and validated diagnostic algorithms are lacking. Therefore, this retrospective cross-sectional study aimed to compare the diagnostic performance of seven commercial antibody assays for LNB diagnosis. Random forest (RF) modeling was conducted to investigate whether the diagnostic performance using the antibody assays could be improved by including several routine cerebrospinal fluid (CSF) parameters (i.e., leukocyte count, total protein, blood-CSF barrier functionality, and intrathecal total antibody synthesis), two-tier serology on serum, the CSF level of the B-cell chemokine (C-X-C motif) ligand 13 (CXCL13), and a Borrelia species PCR on CSF. In total, 156 patients were included who were classified as definite LNB (n = 10), possible LNB (n = 7), or non-LNB patient (n = 139) according to the criteria of the European Federation of Neurological Societies using a consensus strategy for intrathecal Borrelia-specific an...",Retrospective Evaluation of Various Serological Assays and Multiple Parameters for Optimal Diagnosis of Lyme Neuroborreliosis in a Routine Clinical Setting.
821005,"Background/AimsArterial thrombosis and positive antiphospholipid antibodies havebeen reported following COVID-19 infection. We report a case of aorticthrombosis occurring after COVID-19 pneumonitis, with persistentlypositive anticardiolipin antibodies.MethodsA 63-year-old lady presented with sudden onset of abdominal painradiating to the back, associated with vomiting. She reported feelingslightly breathless, and ten days prior to admission a throat swab forSARS-CoV-2 tested positive. Oxygen saturation was 96% (94-100%)on room air. Respiratory examination revealed minimal basal cracklesbilaterally. Her heart sounds were normal, and there was no carotid orsubclavian bruit. Radial, ulnar, brachial, subclavian and temporalpulses were intact and symmetrical. The abdomen was soft, but tenderover the right upper quadrant and epigastrium. Nailfolds were normal, and there was no livedo reticularis and no vasculitic rash. Initialinvestigations showed a CRP of 235 mg/L (&lt;5), Whit

In [17]:
# Keep only the first 20000 rows
limited_df = df_cleaned.iloc[:10000, :]

In [18]:
describe_df(limited_df)


DataFrame shape: (10000, 2)

10,000 samples

Missing Data: 
input     0
output    0
dtype: int64

Duplicates: 0

Categorical Features: 
input, output

DataFrame Head: 



,input,output
0,"INTRODUCTION: Infection with the new coronavirus responsible for Severe Acute Respiratory Syndrome (SARS-CoV-2) continues to spread worldwide. In Brazil, there are already more than 230 thousand dead, many of these older adults. OBJECTIVE: To present the clinical characteristics of older Brazilian adults infected by COVID-19, in the epidemiological weeks (EW) 34-52, and to verify factors responsible for the increased risk of death. METHODS: Retrospective and observational study conducted with secondary publicly available data, provided by the Brazilian Ministry of Health. 1,544 confirmed cases of registered COVID-19 infection were included between August 16 and December 26, 2020, aged 60 or older. OUTCOMES: Demographic data, comorbidity, symptoms for disease, clinical information: days of hospitalization, chest X-ray, type of RT-PCR. RESULTS: 48% of patients admitted to the ICU with evidence for SARS-CoV-2 died. Symptoms and comorbidities related to increased chance of death (OR) w...",Clinical characteristics of 1544 Brazilians aged 60 years and over with laboratory evidence for SARS-CoV-2
1,"OBJECTIVE: To compare maternal and perinatal outcomes between randomized trials and observational studies in which conservative management was performed for more than 48 h in patients with early-onset severe preeclampsia. METHODOLOGY: We searched PubMed, LILACS, Cochrane and Google Scholar. The studies were divided in two groups: randomized and observational studies, from 1990 to 2018 that included patients with severe preeclampsia before 34 weeks of gestation with pregnancy prolongation &#8805;48 h but that did not include fetal growth restriction or HELLP syndrome at the beginning. The main variables recorded were maternal and perinatal complications. MAIN RESULTS: Forty-four studies met the inclusion criteria, and 5 of these were randomized. The average pregnancy prolongation was 9 days, with no difference between groups. Maternal complications were significantly more common in observational studies, RR = 0.71, 95% CI (0.54-0.93), p = .009. Perinatal complications were also sign...",Conservative management of early-onset severe preeclampsia: comparison between randomized and observational studies a systematic review
2,"Background:In recent years, an integrated programme of vector control, screening and treatment of gambiense human African trypanosomiasis (gHAT) infections has led to a rapid decline in cases in the Mandoul disease focus of Chad. In this study, we assess whether elimination of transmission has already been achieved in the region despite low-level case reporting, quantify the role of intensified interventions in transmission reduction, and predict the trajectory of gHAT in Mandoul for the next decade under a range of control scenarios. Method: We utilise human case data (2000-2019) to update a previous model of transmission of gHAT in Mandoul. We also test the updated model, which now has refined assumptions on diagnostic specificity of the current algorithm and an improved fitting method, via a data censoring approach.Results:We conclude that passive detection rates have increased due to improvements in diagnostic availability in fixed health facilities since 2015, by 2.1-fold for ...","Update of transmission modelling and projections of gambiense human African trypanosomiasis in the Mandoul focus, Chad"
3,"Microanatomy of the vast majority of human organs at birth is characterized by marked differences as compared to adult organs, regarding their architecture and the cell types detectable at histology. In preterm neonates, these differences are even more evident, due to the lower level of organ maturation and to ongoing cell differentiation. One of the most remarkable finding in preterm tissues is the presence of huge amounts of stem/progenitor cells in multiple organs, including kidney, brain, heart, adrenals, and lungs. In other organs, such as liver, the completely different burden of cell types in 


DataFrame Tail: 



,input,output
10004,"Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) causes coronavirus disease 2019 (COVID-19), an infectious disease characterized by strong induction of inflammatory cytokines, progressive lung inflammation and potentially multi-organ dysfunction. It remains unclear whether SARS-CoV-2 is sensed by pattern recognition receptors (PRRs) leading to immune activation. Several studies suggest that the Spike (S) protein of SARS-CoV-2 might interact with Toll-like receptor 4 (TLR4) and thereby activate immunity. Here we have investigated the role of TLR4 in SARS-CoV-2 infection and immunity. Neither exposure of isolated S protein, SARS-CoV-2 pseudovirus nor a primary SARS-CoV-2 isolate induced TLR4 activation in a TLR4-expressing cell line. Human monocyte-derived dendritic cells (DCs) express TLR4 but not ACE2, and DCs were not infected by a primary SARS-CoV-2 isolate. Notably, neither S protein nor the primary SARS-CoV-2 isolate induced DC maturation or cytokines, indicating th...",SARS-CoV-2 infection activates dendritic cells via cytosolic receptors rather than extracellular TLRs
10005,"Social networking services played a crucial role in the management of previous outbreaks around the world. African populations are increasingly using social networks and this may have benefits but also harmful consequences, especially at this time of coronavirus disease 2019 pandemic. This paper concisely discusses of these consequences which include the propagation of “fake news” and the misinterpretation of messages pertaining to the prevention and the treatment of the disease. Moreover, our commentary provides some ways to alleviate them, chiefly represented by a framed and practical communication by health authorities. We suggest for instance the systematic sharing of correct messages through official Facebook and Twitter accounts and the conception of tailored web tools dedicated to the verification of circulating information.",The impact of social networking services on the coronavirus disease 2019 (COVID-19) pandemic in sub-Saharan Africa
10006,"We demonstrate that universal scaling behavior is observed in the current coronavirus (COVID-19) spread in various countries. We analyze the numbers of infected people in selected eleven countries (Japan, USA, Russia, Brazil, China, Italy, Indonesia, Spain,South Korea, UK, and Sweden). By using the double exponential function called the Gompertz function, fG(x)=exp(-e-x), the number of infected people is well described as N(t)=N0 fG({gamma}(t-t0)), where N0, {gamma} and t0 are the final total number of infected people, the damping rate of the infection probability and the peak time of dN(t)/dt, respectively. The scaled data of infected people in most of the analyzed countries are found to collapse onto a common scaling function fG(x) with x={gamma}(t-t0) in the range of fG(x) {+/-} 0.05. The recently proposed indicator so-called the K value, the increasing rate of infected people in one week, is also found to show universal behavior. The mechanism for the Gompertz function to appea...",Universality in COVID-19 spread in view of the Gompertz function
10007,"As part of the urgent need to respond to the COVID-19 pandemic, governments, healthcare providers, and businesses have looked to applications of Artificial Intelligence (AI) to compensate for the unavailability of human workers. This interest has renewed the debate regarding the use of AI for the automation of work, which has been described as Intelligent Automation (IA). A new dimension to this debate is whether COVID-19 will be the catalyst for higher IA adoption levels. This article reviews arguments in favour of COVID-19 increasing the level of IA adoption and possible counter-arguments. Key arguments in favour of increased IA adoption include consumer preferences changing to favour IA, increasing familiarity of IA technologies, and increased business confidence in IA. Counter-arguments include big data availabili

In [19]:
# Save DataFrame to a CSV file
limited_df.to_csv('New_medical_meadow_cord19.csv', index=False)

#from google.colab import files

# Download the CSV file
#files.download('New_medical_meadow_cord19.csv')


In [20]:
limited_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 0 to 10008
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input   10000 non-null  object
 1   output  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


# Preprocessing Data

In [21]:
# function for clean tags and empty lines
def clean_tags(text):
    clean = re.compile('<.*?>')
    clean = re.sub(clean, '', text) #Replacing tags text by an empty string

    clean = '\n'.join([line for line in clean.split('\n') if not re.match(r'.*:\s*$',line)]) # Removing empty dialogues

    return clean

In [22]:
# Apply clean tags function for entire dataset
def clean_df(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(clean_tags)
    return df

# Cleaning texts in all datasets
limited_df = clean_df(limited_df,['input', 'output'])

<ipython-input-22-ec7cc2fccddb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna('').apply(clean_tags)


In [23]:
# Assuming `df` is your DataFrame with 20,000 rows

# Split the Dataset
train_val, test_df = train_test_split(limited_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val, test_size=0.25, random_state=42)
# # Check the shape of each dataset
# print("Train set:", train_df.shape)
# print("Validation set:", valid_df.shape)
# print("Test set:", test_df.shape)

# # Verify the row counts
# print(f"Training set: {len(train_df)} rows")
# print(f"Test set: {len(test_df)} rows")
# print(f"Validation set: {len(valid_df)} rows")
# print('\n' * 2)

# # Print first 5 rows from each
# print("First 5 rows of train_df:")
# print(train_df.head(5))

# print("\nFirst 5 rows of test_df:")
# print(test_df.head(5))

# print("\nFirst 5 rows of valid_df:")
# print(valid_df.head(5))


In [24]:
describe_df(train_df)


DataFrame shape: (6000, 2)

6,000 samples

Missing Data: 
input     0
output    0
dtype: int64

Duplicates: 0

Categorical Features: 
input, output

DataFrame Head: 



,input,output
8596,"Traditional selfish routing literature quantifies inefficiency in transportation systems with single-attribute costs using price-of-anarchy (PoA), and provides various technical approaches (e.g. marginal cost pricing) to improve PoA of the overall network. Unfortunately, practical transportation systems have dynamic, multi-attribute costs and the state-of-the-art technical approaches proposed in the literature are infeasible for practical deployment. In this paper, we offer a paradigm shift to selfish routing via characterizing idiosyncratic, multi-attribute costs at boundedly-rational travelers, as well as improving network efficiency using strategic information design. Specifically, we model the interaction between the system and travelers as a Stackelberg game, where travelers adopt multi-attribute logit responses. We model the strategic information design as an optimization problem, and develop a novel approximate algorithm to steer Logit Response travelers towards social welfa...",Maximizing Social Welfare in Selfish Multi-Modal Routing using Strategic Information Design for Quantal Response Travelers
3181,"The COVID-19 pandemic has been and is currently still affecting organizations of all sizes and in many industries, and research still lacks profound insights into the managerial implications of this phenomenon In particular, it is unclear how family firms, which are the economic backbone of most of the countries affected by the pandemic, have adapted to COVID-19 This paper addresses this gap by drawing on a rich body of evidence collected from 90 interviews and secondary data in a longitudinal case study of four German family firms We develop a framework for understanding how family firms adapt to exogenous shocks such as the COVID-19 pandemic and find that the exogenous shock further reinforces the family firm?s resource constraints and the family?s fear of losing their socioemotional wealth These motivational sources, in turn, trigger behavioral changes in both the firm and the family In addition to a temporarily induced short-term orientation, these changes manifest in (pseudo)f...",Exogenous shocks and the adaptive capacity of family firms: exploring behavioral changes and digital technologies in the COVID-19 pandemic
5205,"Historically, economic changes have negatively affected the nursing workforce in Ontario. The trend towards part-time and casual employment emerged from healthcare restructuring in the 1990s. The severe acute respiratory syndrome (SARS) outbreak in 2003 alerted the Ontario government to the issue of part-time and casual nursing. In 2007, the Nursing Graduate Guarantee (NGG), a health human resources employment policy, was developed as a financial incentive for employers to hire and mentor new graduate nurses for a six-month period. The purpose of this study was to examine facilitators and barriers to policy implementation and assess the impact of the NGG on full-time employment and workforce integration of new graduate nurses in Ontario. A mixed-methods approach was used and included surveys, interviews and focus groups. Results indicated that full-time employment of new graduate nurses increased during the study period and that mentorship facilitated workforce integration of new g...",Workforce integration of new graduate nurses: evaluation of a health human resources employment policy.
8897,"The ongoing pandemic COVID-19, caused by SARS-CoV-2, has already resulted in more than 3 million cases and more than 200,000 deaths globally. Significant clinical presentations of COVID-19 include respiratory symptoms and pneumonia. In a minority of patients, extrapulmonary organs (central nervous system, eyes, heart, and gut) are affected, with detection of viral RNA in bodily secretions (stool, tears, and saliva). Infection of such extrapulmonary organs may serve as a reservoir for SARS-CoV-2, representing a potential source of viral shedding after the cessation of respiratory symptom


DataFrame Tail: 



,input,output
8879,"In critical patients with Coronavirus Disease (COVID-19), we investigated the diagnostic value of presepsin in the early diagnosis of superinfection with sepsis, and the effect of antibiotic treatment (AT) in the levels of presepsin and procalcitonin and C-reactive protein. A total of 68 critical patients with sepsis and septic shock in the intensive care unit and 20 outpatients (control group) with COVID-19 were taken. ICU patients (n = 68) were further divided into three groups. C(-)AT(-) had negative blood or tracheal aspirate cultures (C) and not AT on admission to ICU (n = 18), C(-)AT(+) had negative C and AT on admission to intensive care unit (n = 31) and C(+) had positive C (n = 19). Presepsin, procalcitonin, C-reactive protein results were compared between the groups. There were no significant relationships between presepsin levels with sepsis, septic shock, mortality, or length of stay in ICU in patients with COVID-19. For procalcitonin and C-reactive protein levels in C(...",Could presepsin be an alternative marker in the early diagnosis of sepsis in COVID-19?
9835,"Considering that female sexual hormones may modulate the inflammatory response and also exhibit direct effects on the cells of the immune system, herein, we intend to discuss the sex differences and the role of estradiol in modulating the lung and systemic inflammatory response, focusing on its possible application as a treatment modality for SARS-CoV-2 patients COVID-19 patients develop severe hypoxemia early in the course of the disease, which is silent most of the time Small fibrinous thrombi in pulmonary arterioles and a tumefaction of endothelial were observed in the autopsies of fatal COVID-19 cases Studies showed that the viral infection induces a vascular process in the lung, which included vasodilation and endothelial dysfunction Further, the proportions of CD4+ T and CD8+ T lymphocytes were strongly reduced in patients with severe SARS-CoV-2 infection Estradiol is connected with CD4+ T cell numbers and increases T-reg cell populations, affecting immune responses to infect...","17β-Estradiol, a potential ally to alleviate SARS-CoV-2 infection"
5273,"IMPORTANCE: One-third of US residents have trouble paying their medical bills. They often turn to their physicians for help navigating health costs and insurance coverage. OBJECTIVE: To determine whether physicians can accurately estimate out-of-pocket expenses when they are given all of the necessary information about a drug’s price and a patient’s insurance plan. DESIGN, SETTING, AND PARTICIPANTS: This national mail-in survey used a random sample of US physicians. The survey was sent to 900 outpatient physicians (300 each of primary care, gastroenterology, and rheumatology). Physicians were excluded if they were in training, worked primarily for the Veterans Administration or Indian Health Service, were retired, or reported 0% outpatient clinical effort. Analyses were performed from July to December 2020. MAIN OUTCOMES AND MEASURES: In a hypothetical vignette, a patient was prescribed a new drug costing $1000/month without insurance. A summary of her private insurance information...",Accuracy of Physician Estimates of Out-of-Pocket Costs for Medication Filling
9675,"The second and third years of the SARS-CoV-2 pandemic have been marked by the repeated emergence and replacement of variants with genetic and phenotypic distance from the ancestral strains, the most recent examples being Delta and Omicron. Here we describe a hamster contact exposure challenge model to assess protection conferred by vaccination or prior infection against re-infection. We found that 2-doses of self-amplifying RNA vaccine based on the ancestral spike ameliorated weight loss following Delta infection and decreased viral loads, but had minimal effect on Omicron/BA.1 infection. Prior infection with ancestral or Alpha variant was partially protective against Omicron/BA.1 infection, whereas all animals previ

In [25]:
NullRaws = train_df['input'].isnull()
filtered_train = train_df[NullRaws]
filtered_train

,input,output


In [26]:
describe_df(test_df)


DataFrame shape: (2000, 2)

2,000 samples

Missing Data: 
input     0
output    0
dtype: int64

Duplicates: 0

Categorical Features: 
input, output

DataFrame Head: 



,input,output
6258,"The COVID-19 pandemic has revealed a range of disease phenotypes in infected patients with asymptomatic, mild, or severe clinical outcomes, but the mechanisms that determine such variable outcomes remain unresolved. In this study, we identified immunodominant CD8 T-cell epitopes in the spike antigen using a novel TCR-binding algorithm. The predicted epitopes induced robust T-cell activation in unexposed donors demonstrating pre-existing CD4 and CD8 T-cell immunity to SARS-CoV-2 antigen. The T-cell reactivity to the predicted epitopes was higher than the Spike-S1 and S2 peptide pools in the unexposed donors. A key finding of our study is that pre-existing T-cell immunity to SARS-CoV-2 is contributed by TCRs that recognize common viral antigens such as Influenza and CMV, even though the viral epitopes lack sequence identity to the SARS-CoV-2 epitopes. This finding is in contrast to multiple published studies in which pre-existing T-cell immunity is suggested to arise from shared epit...",Immunodominant T-cell epitopes from the SARS-CoV-2 spike antigen reveal robust pre-existing T-cell immunity in unexposed individuals
4689,"Whipple's disease (WD) is a rare infectious disease with a wide clinical spectrum. Associated thrombotic manifestations are not well described in WD, only related to 'stroke-like syndrome'. We present a case of a 39-year-old man with a 1-year history of self-limited episodes of fever, associated with generalised adenopathies and recurrent superficial and deep venous thrombosis events, which have resorted four times despite the anticoagulant treatment. Finally, the patient is diagnosed with WD. Following treatment the patient improved in his general condition, and no more episodes of fever neither thrombosis appeared during a follow-up of more than 3 years.",Recurrent thrombosis as a clinical presentation of Whipple disease.
1732,"Background: Children and young adults were initially reported as largely spared from severe complications of SARS-CoV-2 infection, but the impact to this population has been significant Methods: This observational retrospective cohort study includes 420 symptomatic children and young adults with lab confirmed SARS-CoV-2 infection treated between March 15 and June 16, 2020 at Children's National Hospital in Washington DC We identified and compared cohorts of non-hospitalized (N=324) and hospitalized (N=96) patients, including non-critically ill (N=64) and critically ill hospitalized (N=32) patients Clinical and demographic data were extracted from medical records Results: Of 420 SARS-CoV-2-infected symptomatic patients, 23% required hospitalization, of which 67% were non-critically ill and 33% were critically ill All age groups were represented in the symptomatic cohort, with a median age of 8 6 years Patients &gt; 15 years of age represented 44% of critical care admissions Males an...",Severe COVID-19 in children and young adults in the Washington DC metropolitan region
4747,"Latin America (LATAM) children offer special insight into Severe Acute Respiratory Syndrome Coronavirus 2 (SARS COV2) due to high-risk race/ethnicity, variability in medical resources, diverse socioeconomic background, and numerous involved organ systems. This multinational study of LATAM youth examined the distinguishing features of acute or late multisystem SARS COV2 with versus without cardiac involvement. A consecutive sample of youth 0–18 years old (N = 98;50% male) presenting with multisystem SARS COV2 to 32 centers in 10 Latin American countries participating in a pediatric cardiac multi-imaging society were grouped as with versus without cardiac involvement, defined as abnormal echocardiographic findings or arrhythmia. Collected clinical data were analyzed by Student’s t-test or Fisher’s exact test. Cardiac (N = 48, 50% male) versus no cardiac (N = 50, 50% male) were similar in age; weight; nonrespiratory symptoms; and medical history. The cardiac group had 1 death and symp...",Pediat


DataFrame Tail: 



,input,output
6418,"In the current age of overwhelming information and massive production of textual data on the Web, Event Detection has become an increasingly important task in various application domains Several research branches have been developed to tackle the problem from different perspectives, including Natural Language Processing and Big Data analysis, with the goal of providing valuable resources to support decision-making in a wide variety of fields In this paper, we propose a real-time domain-specific clustering-based event-detection approach that integrates textual information coming, on one hand, from traditional newswires and, on the other hand, from microblogging platforms The goal of the implemented pipeline is twofold: (i) providing insights to the user about the relevant events that are reported in the press on a daily basis;(ii) alerting the user about potentially important and impactful events, referred to as hot events, for some specific tasks or domains of interest The algorith...",Event detection in finance using hierarchical clustering algorithms on news and tweets
8292,"BACKGROUND: Extracorporeal membrane oxygenation (ECMO) is a potential treatment option in critically ill COVID-19 patients suffering from acute respiratory distress syndrome (ARDS) if mechanical ventilation (MV) is insufficient; however, thromboembolic and bleeding events (TEBE) during ECMO treatment still need to be investigated. METHODS: We conducted a retrospective, single-center study including COVID-19 patients treated with ECMO. Additionally, we performed a univariate analysis of 85 pre-ECMO variables to identify factors influencing incidences of thromboembolic events (TEE) and bleeding events (BE), respectively. RESULTS: Seventeen patients were included; the median age was 57 years (interquartile range [IQR]: 51.5-62), 11 patients were males (65%), median ECMO duration was 16 days (IQR: 10.5-22), and the overall survival was 53%. Twelve patients (71%) developed TEBE. We observed 7 patients (41%) who developed TEE and 10 patients (59%) with BE. Upper respiratory tract (URT) b...",Thromboembolic and Bleeding Events in COVID-19 Patients receiving Extracorporeal Membrane Oxygenation
7860,"Supercritical Fluid Chromatography (SFC-UV) employing a carbon dioxide (CO2) and 10 mM ammonium acetate in MeOH-water (95:5 v/v) gradient provides a rapid analysis (tG <10 min) of 31 novel, regioisomeric diphenidine-derived psychoactive substances, on a range of stationary phases of differing polarity. Medium to large selectivity differences between regioisomers, were observed on the acidic, neutral and basic SFC phases. For individual substituted ortho-, meta- and para-isomers, the same elution order was observed irrespective of the nature of the stationary phase. The acidic silica stationary phases yielded longer retention of the diphenidines via electrostatic attraction, whereas the basic phases resulted in shorter retention via electrostatic repulsion. SFC effected baseline separation of seven of the eight substituted groups of ortho-, meta- and para-diphenidines evaluated on a range of stationary phases. A simple silica phase achieved baseline separation of six of the regioiso...",Guilty by dissociation: Part B: evaluation of Supercritical Fluid Chromatography (SFC-UV) for the analysis of regioisomeric diphenidine-derived Novel Psychoactive Substances (NPS).
1095,"Background The SARS-CoV-2 B.1.617.2 (Delta) variant has caused a new surge in the number of COVID-19 cases. The effectiveness of inactivated vaccines against this variant is not fully understood. Methods Using data from a recent large-scale outbreak of B.1.617.2 SARS-COV-2 infection in Jiangsu, China, we conducted a real-world study to explore the effect of inactivated vaccine immunization on the course of disease in patients infected with the Delta variant. Results Out of 476 patients with B.1.617.2 infection, 184 were unvaccinated, 105 were partially vaccinated, and 187 were fully

In [27]:
describe_df(val_df)


DataFrame shape: (2000, 2)

2,000 samples

Missing Data: 
input     0
output    0
dtype: int64

Duplicates: 0

Categorical Features: 
input, output

DataFrame Head: 



,input,output
3191,"OBJECTIVES: To examine functional outcomes of post-acute care for coronavirus disease 2019 (COVID-19) in skilled nursing facilities (SNFs). DESIGN: Retrospective cohort. SETTING AND PARTICIPANTS: Seventy-three community-dwelling adults &#8805;65 years of age admitted for post-acute care from 2 SNFs from March 15, 2020, to May 30, 2020. MEASURE(S): COVID-19 status was determined from chart review. Frailty was measured with a deficit accumulation frailty index (FI), categorized into nonfrail, mild frailty, and moderate-to-severe frailty. The primary outcome was community discharge. Secondary outcomes included change in functional status from SNF admission to discharge, based on modified Barthel index (mBI) and continuous functional scale scored by physical (PT) and occupational therapists (OT). RESULTS: Among 73 admissions (31 COVID-19 negative, 42 COVID-19 positive), mean [standard deviation (SD)] age was 83.5 (8.8) and 42 (57.5%) were female, with mean FI of 0.31 (0.01) with no dif...",Recovery from Coronavirus Disease 2019 among Older Adults in Post-Acute Skilled Nursing Facilities
8300,"PURPOSE Complicated gastric lymphatic drainage potentially undermines the utility of sentinel node (SN) biopsy in patients with gastric cancer. Encouraged by several favorable single-institution reports, we conducted a multicenter, single-arm, phase II study of SN mapping that used a standardized dual tracer endoscopic injection technique. PATIENTS AND METHODS Patients with previously untreated cT1 or cT2 gastric adenocarcinomas 4 cm. We observed no serious adverse effects related to endoscopic tracer injection or the SN mapping procedure. CONCLUSION The endoscopic dual tracer method for SN biopsy was confirmed as safe and effective when applied to the superficial, relatively small gastric adenocarcinomas included in this study.",Sentinel node mapping for gastric cancer: a prospective multicenter trial in Japan.
1710,"To prevent the transmission of COVID-19, the health authorities recommend, in addition to collective protection measures, that all employees wear masks at their workplace. More generally, the general public is encouraged to wear a mask at least in all enclosed spaces. For healthcare workers, surgical masks and FFP2 respirators are recommended depending on the exposure risk. This article defines the different types of mask, how to identify them and how to use each one appropriately in the fight against the transmission of COVID-19.",Using masks against COVID-19
7516,"BACKGROUND AND AIMS EUS-guided liver biopsy is an emerging method of liver tissue acquisition which is safe and had been shown to produce excellent histological yield. There is limited data comparing the diagnostic yield of different FNA needles. We aimed to compare the diagnostic performance of four commercially available 19-gauge FNA needles. METHODS Four FNA needles and one percutaneous needle were used to perform liver biopsies on two human cadaveric livers: Cook Echotip Procore™, Olympus EZ Shot 2™, Boston Scientific Expect Slimline™, Covidien SharkCore™, and an 18-gauge percutaneous needle (TruCore™, Argon Medical Devices). Each needle obtained biopsies by three, six, and nine complete back-and-forth motions of the needle (""throw"") with a fanning technique. The combined lengths of specimen fragments and the total number of complete portal tracts (CPT) were measured by a blinded pathologist. One-way analysis of variance (ANOVA) and Bonferroni correction were used for statistic...",Comparison of the Diagnostic Yield of EUS Needles for Liver Biopsy: Ex Vivo Study.
1461,"In a study exploring the origins of cognitive dissonance, preschoolers and capuchins were given a choice between two equally preferred alternatives (two different stickers and two differently colored M&M's, respectively). On the basis of previous research with adults, this choice was thought to cause dissonance because it conflicted with subjects' belief that the two options w


DataFrame Tail: 



,input,output
2837,"Introduction: Avermectins are common antiparasitic drugs, derived from Streptomyces bacteria that exhibit activity against arthropods and nematodes. Ivermectin, an avermectin derivative, is used as a treatment for parasitic infections in humans and domesticated animals.Discussion: Ivermectin's mechanism of action involves binding to ligand-gated ion channel receptors including glutamate, GABA, and glycine, resulting in parasitic paralysis and death. Due to varying expression of these ion channel receptors in vertebrate species, ivermectin toxicity is rarely reported in mammals. Ivermectin is also a substrate for P-glycoprotein, which limits its neurological toxicity in humans. Genetic polymorphisms in P-glycoprotein or coadministration of P-glycoprotein inhibitors may increase the neurotoxicity of ivermectin. Other toxic effects of ivermectin after therapeutic oral use include edema, rash, headache, and ocular complaints. Most of these effects are mild and short in duration. Iverme...",Ivermectin: a mini-review
449,"In this paper we look beyond metrics-based evaluation of Information Retrieval systems, to explore the reasons behind ranking results. We present the content-focused Neural-IR-Explorer, which empowers users to browse through retrieval results and inspect the inner workings and fine-grained results of neural re-ranking models. The explorer includes a categorized overview of the available queries, as well as an individual query result view with various options to highlight semantic connections between query-document pairs. The Neural-IR-Explorer is available at: https://neural-ir-explorer.ec.tuwien.ac.at/.",Neural-IR-Explorer: A Content-Focused Tool to Explore Neural Re-ranking Results
6692,"PURPOSE: Previous studies have outlined mechanisms by which Mycoplasma pneumonia (M. pneumonia) infection may promote allergic lung inflammation and airway remodeling, and increasing evidence from human studies suggests that atypical bacterial infections contribute to asthma exacerbation, chronic asthma, and disease severity with changes in cytokine expression. The present study evaluated changes in serum levels of vascular endothelial growth factor (VEGF) and interleukin (IL)-5 in atopic children with Mycoplasma pneumoniae pneumonia. METHODS: We recruited a total of 72 children with pneumonia. The patients were divided into 4 groups: atopic children with M. pneumonia pneumonia (group I, n=24), non-atopic children with M. pneumonia pneumonia (group II, n=23), atopic children with viral pneumonia (group III, n=13), and non-atopic children with viral pneumonia (group IV, n=12). Serum levels of IL-5, IL-13, VEGF, and tumor necrosis factor-α were measured at admission and at recovery u...",Mycoplasma pneumoniae Infection Affects the Serum Levels of Vascular Endothelial Growth Factor and Interleukin-5 in Atopic Children
3564,"Despite remarkable success in deep learning-based face-related models, these models are still limited to the domain of real human faces. On the other hand, the domain of animation faces has been studied less intensively due to the absence of a well-organized dataset. In this paper, we present a large-scale animation celebfaces dataset (AnimeCeleb) via controllable synthetic animation models to boost research on the animation face domain. To facilitate the data generation process, we build a semi-automatic pipeline based on an open 3D software and a developed annotation system. This leads to constructing a large-scale animation face dataset that includes multi-pose and multi-style animation faces with rich annotations. Experiments suggest that our dataset is applicable to various animation-related tasks such as head reenactment and colorization.",AnimeCeleb: Large-Scale Animation CelebFaces Dataset via Controllable 3D Synthetic Models
9604,"We examined the documentation underlying the decision to permit the Southern District Health Board (SDHB) to join the National Bowel Screening Programme (NB

In [28]:
# function for clean tags and empty lines
def clean_tags(text):
    clean = re.compile('<.*?>')
    clean = re.sub(clean, '', text) #Replacing tags text by an empty string

    clean = '\n'.join([line for line in clean.split('\n') if not re.match(r'.*:\s*$',line)]) # Removing empty dialogues

    return clean

# Function to clean text
def remove_unwanted_text(text):
    return re.sub(r"\[.*?\]|\(.*?\)", "", text)

#function to decode html
def decode_html(text):
    return html.unescape(text)

# Function to remove URLs using regex
def remove_links(text):
    return re.sub(r'http\S+', '', text)

In [29]:
# Apply clean tags function for entire dataset
def clean_df(df, cols):
    for col in cols:
        df[col] = df[col].fillna('').apply(clean_tags)
        df[col] = df[col].fillna('').apply(remove_unwanted_text)
        df[col] = df[col].fillna('').apply(decode_html)
        df[col] = df[col].fillna('').apply(remove_links)
    return df

# Cleaning texts in all datasets
train_df = clean_df(train_df,['input', 'output'])
test_df = clean_df(test_df,['input', 'output'])
val_df = clean_df(val_df,['input', 'output'])

In [30]:
# train_df[['input','output']] # Access the 'input' and 'output' columns
# print("\n"*2)
# test_df[['input','output']] # Access the 'input' and 'output' columns
# print("\n"*2)
# valid_df[['input','output']] # Access the 'input' and 'output' columns

In [31]:
# Transforming dataframes into datasets
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

# Visualizing results
print(train_ds)
print('\n' * 2)
print(test_ds)
print('\n' * 2)
print(val_ds)

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 6000
})



Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 2000
})



Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 2000
})


In [32]:
train_ds[0]

{'input': 'Traditional selfish routing literature quantifies inefficiency in transportation systems with single-attribute costs using price-of-anarchy , and provides various technical approaches  to improve PoA of the overall network. Unfortunately, practical transportation systems have dynamic, multi-attribute costs and the state-of-the-art technical approaches proposed in the literature are infeasible for practical deployment. In this paper, we offer a paradigm shift to selfish routing via characterizing idiosyncratic, multi-attribute costs at boundedly-rational travelers, as well as improving network efficiency using strategic information design. Specifically, we model the interaction between the system and travelers as a Stackelberg game, where travelers adopt multi-attribute logit responses. We model the strategic information design as an optimization problem, and develop a novel approximate algorithm to steer Logit Response travelers towards social welfare using strategic Informa

In [33]:
val_ds[0]

{'input': 'OBJECTIVES: To examine functional outcomes of post-acute care for coronavirus disease 2019  in skilled nursing facilities . DESIGN: Retrospective cohort. SETTING AND PARTICIPANTS: Seventy-three community-dwelling adults ≥65 years of age admitted for post-acute care from 2 SNFs from March 15, 2020, to May 30, 2020. MEASURE: COVID-19 status was determined from chart review. Frailty was measured with a deficit accumulation frailty index , categorized into nonfrail, mild frailty, and moderate-to-severe frailty. The primary outcome was community discharge. Secondary outcomes included change in functional status from SNF admission to discharge, based on modified Barthel index  and continuous functional scale scored by physical  and occupational therapists . RESULTS: Among 73 admissions , mean  age was 83.5  and 42  were female, with mean FI of 0.31  with no differences by COVID-19 status. The mean length of SNF stay for rehabilitation was 21.2 days  for COVID-19 negative with 20  

In [34]:
test_ds[0]

{'input': 'The COVID-19 pandemic has revealed a range of disease phenotypes in infected patients with asymptomatic, mild, or severe clinical outcomes, but the mechanisms that determine such variable outcomes remain unresolved. In this study, we identified immunodominant CD8 T-cell epitopes in the spike antigen using a novel TCR-binding algorithm. The predicted epitopes induced robust T-cell activation in unexposed donors demonstrating pre-existing CD4 and CD8 T-cell immunity to SARS-CoV-2 antigen. The T-cell reactivity to the predicted epitopes was higher than the Spike-S1 and S2 peptide pools in the unexposed donors. A key finding of our study is that pre-existing T-cell immunity to SARS-CoV-2 is contributed by TCRs that recognize common viral antigens such as Influenza and CMV, even though the viral epitopes lack sequence identity to the SARS-CoV-2 epitopes. This finding is in contrast to multiple published studies in which pre-existing T-cell immunity is suggested to arise from shar

# Modeling

In [35]:
from transformers import pipeline

# Loading summarization pipeline with the bart-large-cnn model using PyTorch
summarizer = pipeline('summarization', model = 'facebook/bart-large-xsum')


# Use a pipeline as a high-level helper
#from transformers import pipeline

#summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [36]:
text = """Hugging Face's transformers library provides many tools for modern NLP tasks.
          With pipelines, you can easily load models and perform complex tasks like
          text generation, summarization, or translation with minimal code."""

news = '''Bobi, the world’s oldest dog ever, has died after reaching the almost inconceivable age of 31 years and 165 days, said Guinness World Records (GWR) on Monday.
His death at an animal hospital on Friday was initially announced by veterinarian Dr. Karen Becker.
She wrote on Facebook that “despite outliving every dog in history, his 11,478 days on earth would never be enough, for those who loved him.”
There were many secrets to Bobi’s extraordinary old age, his owner Leonel Costa told GWR in February. He always roamed freely, without a leash or chain, lived in a “calm, peaceful” environment and ate human food soaked in water to remove seasonings, Costa said.
He spent his whole life in Conqueiros, a small Portuguese village about 150 kilometers (93 miles) north of the capital Lisbon, often wandering around with cats.
Bobi was a purebred Rafeiro do Alentejo – a breed of livestock guardian dog – according to his owner. Rafeiro do Alentejos have a life expectancy of about 12-14 years, according to the American Kennel Club.
But Bobi lived more than twice as long as that life expectancy, surpassing an almost century-old record to become the oldest living dog and the oldest dog ever – a title which had previously been held by Australian cattle-dog Bluey, who was born in 1910 and lived to be 29 years and five months old.
However, Bobi’s story almost had a different ending.
When he and his three siblings were born in the family’s woodshed, Costa’s father decided they already had too many animals at home.
Costa and his brothers thought their parents had taken all the puppies away to be destroyed. However, a few sad days later, they found Bobi alive, safely hidden in a pile of logs.
The children hid the puppy from their parents and, by the time Bobi’s existence became known, he was too old to be put down and went on to live his record-breaking life.
His 31st birthday party in May was attended by more than 100 people and a performing dance troupe, GWR said.
His eyesight deteriorated and walking became harder as Bobi grew older but he still spent time in the backyard with the cats, rested more and napped by the fire.
“Bobi is special because looking at him is like remembering the people who were part of our family and unfortunately are no longer here, like my father, my brother, or my grandparents who have already left this world,” Costa told GWR in May. “Bobi represents those generations.”
'''



summary1 = summarizer(text)
summary2 = summarizer(news)

print("\n"*2)
print(summary1)
print("\n"*2)
print(summary2)





[{'summary_text': 'If you are working with natural language processing (NLP), you need to know how to build pipelines.'}]



[{'summary_text': 'The world’s oldest dog has died, Guinness World Records has confirmed.'}]


In [37]:
model_name = 'facebook/bart-large-xsum'                  # model name
tokenizer = BartTokenizer.from_pretrained(model_name)    # Loading Tokanizer

model = BartForConditionalGeneration.from_pretrained(model_name)


# Load model directly
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
#model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [38]:
# function for preprocess datasets
def preprocess_function(examples):
    inputs = [doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["output"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [39]:
# Applying preprocess_function to the datasets
tokenized_train = train_ds.map(preprocess_function, batched=True,
                               remove_columns=['input', 'output', '__index_level_0__']) # Removing features

tokenized_test = test_ds.map(preprocess_function, batched=True,
                               remove_columns=['input', 'output', '__index_level_0__']) # Removing features

tokenized_val = val_ds.map(preprocess_function, batched=True,
                               remove_columns=['input', 'output', '__index_level_0__']) # Removing features

# Printing results
print('\n' * 3)
print('Preprocessed Training Dataset:\n')
print(tokenized_train)
print('\n' * 2)
print('Preprocessed Test Dataset:\n')
print(tokenized_test)
print('\n' * 2)
print('Preprocessed Validation Dataset:\n')
print(tokenized_val)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]





Preprocessed Training Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 6000
})



Preprocessed Test Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})



Preprocessed Validation Dataset:

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
})


In [40]:
# Selecting a sample from the dataset
sample = tokenized_train[0]

# Printing its features
print("input_ids:")                    # These are the token IDs mapped to the dialogues
print(sample['input_ids'])
print("\n")
print("attention_mask:")               # indicates which tokens the model should pay attention to and which tokens should be ignored
print(sample['attention_mask'])
print("\n")
print("sample:")                       # token IDs obtained from the words and subwords in the summaries
print(sample['labels'])
print("\n")

input_ids:
[0, 45461, 20462, 33819, 13144, 24934, 10687, 11, 33939, 11, 4264, 1743, 19, 881, 12, 49202, 1042, 634, 425, 12, 1116, 12, 260, 37208, 2156, 8, 1639, 1337, 3165, 8369, 1437, 7, 1477, 6002, 250, 9, 5, 1374, 1546, 4, 6802, 6, 7708, 4264, 1743, 33, 6878, 6, 3228, 12, 49202, 1042, 8, 5, 194, 12, 1116, 12, 627, 12, 2013, 3165, 8369, 1850, 11, 5, 13144, 32, 4047, 29358, 4748, 13, 7708, 9737, 4, 96, 42, 2225, 6, 52, 904, 10, 28323, 3294, 7, 20462, 33819, 1241, 2048, 2787, 39885, 5183, 6, 3228, 12, 49202, 1042, 23, 8191, 38441, 12, 43198, 11774, 6, 25, 157, 25, 3927, 1546, 5838, 634, 3461, 335, 1521, 4, 19282, 6, 52, 1421, 5, 10405, 227, 5, 467, 8, 11774, 25, 10, 312, 1043, 5576, 2865, 177, 6, 147, 11774, 7581, 3228, 12, 49202, 7425, 405, 8823, 4, 166, 1421, 5, 3461, 335, 1521, 25, 41, 25212, 936, 6, 8, 2179, 10, 5808, 32161, 17194, 7, 16452, 9359, 405, 19121, 11774, 1567, 592, 6642, 634, 3461, 3522, 1521, 479, 166, 8085, 5, 819, 9, 5463, 6934, 15, 10, 22781, 4670, 1546, 19, 3228, 1

In [41]:
# # Remove unnecessary columns from the tokenized dataset
# columns_to_remove = ['__index_level_0__',' input',' output']
# train_ds.remove_columns(columns_to_remove)
# valid_ds.remove_columns(columns_to_remove)
# test_ds.remove_columns(columns_to_remove)

In [42]:
# Instantiating Data Collator to batch the data
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [43]:
!pip install rouge_score # Install the missing rouge_score package.
import evaluate          # Make sure to import the evaluate library if you haven't already.

# Loading ROUGE Score
metric = evaluate.load('rouge')

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=823a52bd43dadde74fcdbb5384b7e157ef43fe765343c6cdb32ce5400fb6f4b7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [44]:
import numpy as np
import nltk

def compute_metrics(eval_pred):
    predictions, labels = eval_pred  # Obtaining predictions and true labels

    # Decoding predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Handling masked tokens in labels (i.e., label = -100)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute the Rouge score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Check if `result` has the expected structure, and safely access fmeasure
    if isinstance(result, dict):
        result = {key: (value['fmeasure'] * 100 if isinstance(value, dict) and 'fmeasure' in value else value)
                  for key, value in result.items()}
    else:
        print("Unexpected result structure:", result)

    # Add mean-generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [45]:
# Defining parameters for training
training_args = Seq2SeqTrainingArguments(
    output_dir = 'Bart_Health',
    evaluation_strategy = "steps",
    eval_steps = 500,
    save_strategy = 'steps',
    save_steps=500,
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss',
    seed = 42,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
# Defining Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [47]:
#!pip install accelerate==0.15.0 transformers==4.28.1

In [48]:
#print(len(train_df)) # Check the length of your training DataFrame

# Check for any inconsistencies or missing indices in the DataFrame
#print(train_df.index.min())
#print(train_df.index.max())
#print(train_df.index.is_monotonic_increasing)

In [49]:
#train_df = train_df.reset_index(drop=True)

In [50]:
# Training model
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,2.079600,2.035082,0.470600,0.269100,0.400000,0.400600,22.871000
1000,1.725600,2.095560,0.477300,0.268900,0.399700,0.400200,25.190500
1500,1.527400,2.034462,0.474200,0.264000,0.396400,0.396700,27.458500
2000,1.177300,2.142511,0.477000,0.266600,0.398400,0.398900,25.600500
2500,1.083800,2.146353,0.476800,0.265900,0.398100,0.399000,25.230000
3000,0.966900,2.180473,0.478000,0.265300,0.396800,0.397300,26.184000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a G

TrainOutput(global_step=3000, training_loss=1.4267721455891926, metrics={'train_runtime': 6401.0527, 'train_samples_per_second': 3.749, 'train_steps_per_second': 0.469, 'total_flos': 2.0613084177924096e+16, 'train_loss': 1.4267721455891926, 'epoch': 4.0})

In [53]:
# Evaluating model performance on the tokenized validation dataset
validation = trainer.evaluate(eval_dataset = tokenized_val)

print(validation)

{'eval_loss': 2.0773138999938965, 'eval_rouge1': 0.4651, 'eval_rouge2': 0.2552, 'eval_rougeL': 0.3867, 'eval_rougeLsum': 0.3876, 'eval_gen_len': 27.5575, 'eval_runtime': 697.7891, 'eval_samples_per_second': 2.866, 'eval_steps_per_second': 0.717, 'epoch': 4.0}


In [54]:
# Saving model
directory = "BART_Finetuned_Medical"
trainer.save_model(directory)

# Saving model tokenizer
tokenizer.save_pretrained(directory)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


('BART_Finetuned_Medical/tokenizer_config.json',
 'BART_Finetuned_Medical/special_tokens_map.json',
 'BART_Finetuned_Medical/vocab.json',
 'BART_Finetuned_Medical/merges.txt',
 'BART_Finetuned_Medical/added_tokens.json')

In [61]:
from huggingface_hub import notebook_login

notebook_login()

In [63]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from huggingface_hub import HfApi, HfFolder, Repository

model_name = "Kulanperera/bart_samsum_medical_trained_model"

model = AutoModelForSeq2SeqLM.from_pretrained("BART_Finetuned_Medical")
tokenizer = AutoTokenizer.from_pretrained("BART_Finetuned_Medical")

model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 62, 'min_length': 11, 'early_stopping': True, 'num_beams': 6, 'no_repeat_ngram_size': 3, 'forced_eos_token_id': 2}


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kulanperera/bart_samsum_medical_trained_model/commit/7cca732c104aaebd568af606076289c34bb19bbb', commit_message='Upload tokenizer', commit_description='', oid='7cca732c104aaebd568af606076289c34bb19bbb', pr_url=None, pr_revision=None, pr_num=None)